# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2019 Semester 1
-----
## Project 1: Gaining Information about Naive Bayes
-----
###### Student Name(s): Akira and Callum
###### Python version: 3.7.1 from Anaconda 
###### Submission deadline: 1pm, Fri 5 Apr 2019

This iPython notebook is a template which you may use for your Project 1 submission. (You are not required to use it; in particular, there is no need to use iPython if you do not like it.)

Marking will be applied on the five functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
import pandas as pd
from IPython.display import display, Latex
from collections import *
import numpy as np

# POSSIBLE CSVs
# hypothyroid.csv seems like an easy choice for now
d1 =  'anneal.csv'
h1 = 'family,product-type,steel,carbon,hardness,temper_rolling,condition,formability,strength,non-ageing,surface-finish,surface-quality,enamelability,bc,bf,bt,bw-me,bl,m,chrom,phos,cbond,marvi,exptl,ferro,corr,bbvc,lustre,jurofm,s,p,shape,oil,bore,packing,class'.split(',')

d2 =  'breast-cancer.csv'
h2 = 'age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,class'.split(',')

d3 =  'car.csv'
h3 = 'buying,maint,doors,persons,lug_boot,safety,class'.split(',')

d4 =  'cmc.csv'
h4 = 'w-education,h-education,n-child,w-relation,w-work,h-occupation,standard-of-living,media-exposure,class'.split(',')

d5 =  'hepatitis.csv'
h5 = 'sex,steroid,antivirals,fatigue,malaise,anorexia,liver-big,liver-firm,spleen-palpable,spiders,ascites,varices,histology,class'.split(',')

d6 =  'hypothyroid.csv'
h6 = 'sex,on-thyroxine,query-on-thyroxine,on_antithyroid,surgery,query-hypothyroid,query-hyperthyroid,pregnant,sick,tumor,lithium,goitre,TSH,T3,TT4,T4U,FTI,TBG,class'.split(',')

d7 =  'mushroom.csv'
h7 = 'cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class'.split(',')

d8 =  'nursery.csv'
h8 = 'parents,has_nurs,form,children,housing,finance,social,health,class'.split(',')

d9 = 'primary-tumor.csv'
h9 = 'age,sex,histologic-type,degree-of-diffe,bone,bone-marrow,lung,pleura,peritoneum,liver,brain,skin,neck,supraclavicular,axillar,mediastinum,abdominal,class'.split(',')

datasets = [d1,d2,d3,d4,d5,d6,d7,d8,d9]
dataset_headers = [h1,h2,h3,h4,h5,h6,h7,h8,h9]

# Dictionary that holds key 'filename' and values 'attribute names'
dictionary = {}

for i in range(9):
    dictionary[datasets[i]] = dataset_headers[i]
    
# Function that finds the corresponding attribute names
def set_column(filename):
    return dictionary[filename]

In [4]:
# Function I ripped off my previous ETL scripts, slightly modified
def describe(filename):
    df = pd.read_csv(filename, header = None, names = set_column(filename))
    
    print("**************************************************************************")
    print(f'NAME OF FILE: {filename}')
    print("**************************************************************************")
    
    # General description for dataset
    print(f"Number of rows: {len(df)}")
    print(f"Number of attributes/columns: {len(df.columns)}")
    print(f"Column names: [{', '.join(list(df.columns))}]")
    print(f"Column datatypes: [{', '.join(set(list(df.dtypes.astype(str))))}]")
    print(f"Shape of dataset: {(df.shape,)}\n")
    
    print("**************************************************************************")

    # Double check for missing values
    if (df.replace('-', np.NaN).isnull().sum().sum() == 0):
        print("No missing values and no imputations required.\n")
    else:
        print(f"Number of missing values = {df.replace('-', np.NaN).isnull().sum().sum()}")
        print("Missing values and we require imputations.\n")
        print("Using Average:")
        display(df.mean())
        print("\nUsing Median:")
        display(df.median())
    
    print("**************************************************************************")
        
    print("Number of unique values per attribute:")
    for i in df.columns:
        print(f"{i}: {len(df[i].unique())}")
    
    print("**************************************************************************")
    
    verify_prob = 0
    
    # MODIFIED SECTION
    print('Prior Probabilities:')
          
    priors = {}
    for i in df['class'].unique():    
        
        # Number of class labels / number of instances
        prob = len(df.loc[df['class'] == i]) / len(df)
        
        verify_prob += prob
        
        print(f"Pr({i}) = {prob}")
        
        priors[i] = prob
        
    # Just in case we break any axioms
    if verify_prob != 1:
        print('You just broke an axiom. Sum of all probs != 1')
        
        return -1
          
    print("**************************************************************************")
    display(df.head())

In [5]:
# This function should open a data file in csv, and transform it into a usable format 
def preprocess(filename):
    # Read in csv and add the column header
    df = pd.read_csv(filename, header = None, names = set_column(filename))
    N = len(df)
    
    # TEST, CROSS-VALIDATION (cv), TRAIN
    # I'm thinking about 3:1:1 according to Andrew Ng
    ratio = round(N*0.2)
    
    return_df = df.copy()
    
    cv = df.sample(ratio)
    cv_labels = cv['class']
    cv.drop('class', axis=1, inplace=True)
    
    df = df.drop(cv.index)
    
    test = df.sample(ratio)
    test_labels = test['class']
    test.drop('class', axis=1, inplace=True)
    
    train = df.drop(test.index)
    
    # Verify that the split works
    print("Split Success:", len(cv) + len(test) + len(train) == N)
    
    # not sure how you would return the three dataframes seperately
    # for now return the whole thing
    return return_df

train_dataset = preprocess('hypothyroid.csv')

Split Success: True


In [7]:
# This function should build a supervised NB model
def train(train_dataset):
    priors = {}
    posteriors = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))
    
    df = train_dataset
    
    for label in df['class'].unique():    
        # Number of class labels / number of instances
        N = len(df)
        prob = len(df.loc[df['class'] == label]) / N
        
        # Dictionary of Prior Probabilities
        # print(f"Pr({label}) = {prob}")
        priors[label] = prob
        
        for attribute in df.columns[:-1]:
            # for each attribute given the class
            df1 = df.loc[df['class'] == label, [attribute,'class']]
            
            # Number of instances corresponding to attribute given class
            n = len(df1)
            
            # REPLACED WITH COUNTER
            count = Counter(df1[attribute])
            ### display(count)
            for i in count:
                posteriors[label][attribute][i] = count[i] / n
    
    # Uncomment to see the sin I have created
    print(posteriors)
    
    #########################################################################
    # Let say I wanted Pr(Sex = M | hypothyroid) and Pr(Sex = M | negative)
    # Access using dict[class label][attribute][attribute value]
    
    # print("Pr(Sex = M | hypothyroid)=",posteriors['hypothyroid']['sex']['M'])
    # print("Pr(Sex = M | negative)=",posteriors['negative']['sex']['M'])    
    
    # All posteriors
    for class_label in posteriors:
        print(f"\nPr({class_label}) = {prob}")
        for attribute in posteriors[class_label]:
            for val in posteriors[class_label][attribute]:
                print(f"Pr({attribute} = {class_label} | class = {class_label}) = {posteriors[class_label][attribute][val]}")
    #########################################################################
    return

train(train_dataset)

defaultdict(<function train.<locals>.<lambda> at 0x000002A57B48EEA0>, {'hypothyroid': defaultdict(<function train.<locals>.<lambda>.<locals>.<lambda> at 0x000002A57BA992F0>, {'sex': defaultdict(<class 'float'>, {'M': 0.25165562913907286, 'F': 0.7350993377483444, '?': 0.013245033112582781}), 'on-thyroxine': defaultdict(<class 'float'>, {'f': 0.9072847682119205, 't': 0.09271523178807947}), 'query-on-thyroxine': defaultdict(<class 'float'>, {'f': 1.0}), 'on_antithyroid': defaultdict(<class 'float'>, {'f': 0.9933774834437086, 't': 0.006622516556291391}), 'surgery': defaultdict(<class 'float'>, {'f': 0.9337748344370861, 't': 0.06622516556291391}), 'query-hypothyroid': defaultdict(<class 'float'>, {'f': 0.8675496688741722, 't': 0.13245033112582782}), 'query-hyperthyroid': defaultdict(<class 'float'>, {'f': 0.9536423841059603, 't': 0.046357615894039736}), 'pregnant': defaultdict(<class 'float'>, {'f': 0.9933774834437086, 't': 0.006622516556291391}), 'sick': defaultdict(<class 'float'>, {'f': 

In [6]:
# This function should predict the class for an instance or a set of instances, based on a trained model 
def predict():
    return

In [7]:
# This function should evaluate a set of predictions, in a supervised context 
def evaluate():
    return

In [8]:
# This function should calculate the Information Gain of an attribute or a set of attribute, with respect to the class


# ENTROPY
def info_gain():
    return

Questions (you may respond in a cell or cells below):

1. The Naive Bayes classifiers can be seen to vary, in terms of their effectiveness on the given datasets (e.g. in terms of Accuracy). Consider the Information Gain of each attribute, relative to the class distribution — does this help to explain the classifiers’ behaviour? Identify any results that are particularly surprising, and explain why they occur.
2. The Information Gain can be seen as a kind of correlation coefficient between a pair of attributes: when the gain is low, the attribute values are uncorrelated; when the gain is high, the attribute values are correlated. In supervised ML, we typically calculate the Infomation Gain between a single attribute and the class, but it can be calculated for any pair of attributes. Using the pair-wise IG as a proxy for attribute interdependence, in which cases are our NB assumptions violated? Describe any evidence (or indeed, lack of evidence) that this is has some effect on the effectiveness of the NB classifier.
3. Since we have gone to all of the effort of calculating Infomation Gain, we might as well use that as a criterion for building a “Decision Stump” (1-R classifier). How does the effectiveness of this classifier compare to Naive Bayes? Identify one or more cases where the effectiveness is notably different, and explain why.
4. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy. How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)
5. Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the Naive Bayes classifier? Explain why, or why not.
6. Naive Bayes is said to elegantly handle missing attribute values. For the datasets with missing values, is there any evidence that the performance is different on the instances with missing values, compared to the instances where all of the values are present? Does it matter which, or how many values are missing? Would a imputation strategy have any effect on this?

Don't forget that groups of 1 student should respond to question (1), and one other question of your choosing. Groups of 2 students should respond to question (1) and question (2), and two other questions of your choosing. Your responses should be about 150-250 words each.

4. Should improve since a using the training dataset to test a supervised algo is like a cheat. the supervised algo has already seen the answers and will recognise it. Using a test / cross-validation will ensure that we are left with unseen data albeit a decrease in training instances

6. performance wise there should be no difference. there is no need to impute as a missing value could be significant in implying something. 